In [27]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats
import pickle
import compound_splitter

dataset_dir = '../En_De_Dataset/All/RefExp'
success_dir = '../En_De_Dataset/All/Success'

In [ ]:

def data_values_move(refexps_dir, success_dir, compound = True):
    splits = compound_splitter.load_dict('../de_lower.dict')
    move_level_data = defaultdict(dict)
    success_types = set()
    for dialogue in listdir(refexps_dir):
        #extract dialogue type
        success_file = dialogue.replace('refexps', 'movesuccess')
        success_file = path.join(success_dir, success_file)
        if dialogue.startswith('FTT'):
            language = 'german'
        else:
            language = 'english'
        dialogue_path = path.join(refexps_dir, dialogue)
        #load referring expressions data
        dialogue_path = refexps.load_data(dialogue_path)
        success_moves = refexps.load_data(success_file)
        #move level
        dialogue_move = refexps.move_level(dialogue_path)
        #iterate over moves
        for move in dialogue_move:
            move_id = dialogue + str(move[0])
            success = success_moves[move[0]]
            refs = move[1]
            if success == 'success':
                success = 'correct'
            if success == 'wrong':
                success = 'incorrect'
            if success == 'na':
                success = 'no_video'
            success_types.add(success)
            if success == 'no_video':
                pass
            else:
                #iterate over objects
                for obj in refs:
                    #iterate over objects
                    refobj = refs[obj]
                    #SIMILARITY OF LEXICAL CONTENT INTO
                    sim_align = []
                    sim_align_speaker = defaultdict(list)
                    sim_coherence_speaker = defaultdict(list)
                    sim_coherence = []
                    for ref in refobj:
                        speaker = ref[1]
                        sim = refexps.lex_sim(ref, refexps.get_previous(ref, refobj), language, splits, compound)
                        if sim != 'undefined':
                            sim_align.append(sim)
                            sim_align_speaker[speaker].append(sim)
                    sim_align = np.mean(sim_align)
                    comb_ref = combinations(refobj,2)
                    for comb in comb_ref:
                        if comb[0] != comb[1]:
                            sim = refexps.lex_sim(comb[0], comb[1], language, splits, compound)
                            if sim != 'undefined':
                                sim_coherence.append(sim)
                                sim_coherence_speaker[comb[0][1]].append(sim)
                                sim_coherence_speaker[comb[1][1]].append(sim)
                    sim_coherence = np.mean(sim_coherence)
                    #movelevel coherence and alignment (for each object)
                    if not np.isnan(sim_coherence):
                        move_level_data['coherence'][move_id] = (sim_coherence, success)
                    if not np.isnan(sim_align):
                        move_level_data['alignment'][move_id] = (sim_align, success)
                    for s in sim_align_speaker:
                        if not np.isnan(np.mean(sim_align_speaker[s])):
                            move_level_data['alignment '+s.replace('-utts', '')][move_id] = (np.mean(sim_align_speaker[s]), success )
                    for s in sim_coherence_speaker:
                        if not np.isnan(np.mean(sim_coherence_speaker[s])) :
                            move_level_data['coherence '+s.replace('-utts', '')][move_id] = (np.mean(sim_coherence_speaker[s]), success)
    return move_level_data

move_level_data = data_values_move(dataset_dir, success_dir)
data_regression = {}
for v in move_level_data:
    data_regression[v] = move_level_data[v].values()
pickle.dump(data_regression, open('regression_lexical.p', 'w'))